In [0]:
from huggingface_hub import notebook_login

# Login to Huggingface to get access to the model
notebook_login()

In [0]:
%pip install xformers

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.5 MB/s eta 0:00:00
     

In [0]:
dbutils.library.restartPython()

In [0]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import os

# it is suggested to pin the revision commit hash and not change it for reproducibility because the uploader might change the model afterwards; you can find the commmit history of Llama-2-13b-chat-hf in https://huggingface.co/meta-llama/Llama-2-13b-chat-hf/commits/main
model = "meta-llama/Llama-2-13b-chat-hf"
revision = "4021a3b5608262f386b2bee683b6348e9228325d"

os.environ["CUDA_VISIBLE_DEVICES"]="0"
device=torch.device("cuda")
tokenizer = AutoTokenizer.from_pretrained(model, padding_side="left")
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    revision=revision,
    return_full_text=False
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [0]:
pipeline.tokenizer.pad_token_id = tokenizer.eos_token_id

# COMMAND ----------

# Define prompt template, the format below is from: http://fastml.com/how-to-train-your-own-chatgpt-alpaca-style-part-one/

# Prompt templates as follows could guide the model to follow instructions and respond to the input, and empirically it turns out to make Falcon models produce better responses

INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)


In [0]:
def gen_text(prompts, use_template=False, **kwargs):
    print(prompts)
    if use_template:
        full_prompts = [
            PROMPT_FOR_GENERATION_FORMAT.format(instruction=prompt)
            for prompt in prompts
        ]
    else:
        full_prompts = prompts

    if "batch_size" not in kwargs:
        kwargs["batch_size"] = 1
    
    # the default max length is pretty small (20), which would cut the generated output in the middle, so it's necessary to increase the threshold to the complete response
    if "max_new_tokens" not in kwargs:
        kwargs["max_new_tokens"] = 1000

    # configure other text generation arguments, see common configurable args here: https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.GenerationConfig
    kwargs.update(
        {
            "pad_token_id": tokenizer.eos_token_id,  # Hugging Face sets pad_token_id to eos_token_id by default; setting here to not see redundant message
            "eos_token_id": tokenizer.eos_token_id,
        }
    )

    outputs = pipeline(full_prompts, **kwargs)
    print(outputs)
    #outputs = [out[0]["generated_text"] for out in outputs]
    out=outputs[0]["generated_text"]

    return out


In [0]:
val = input("Enter the command in hindi: ")
results = gen_text(val)
print(results)

Enter the command in hindi:  एचडीएफसी बैंक में खाता खोलने की प्रक्रिया क्या है? हिंदी में उत्तर दीजिए

एचडीएफसी बैंक में खाता खोलने की प्रक्रिया क्या है? हिंदी में उत्तर दीजिए


/databricks/python/lib/python3.10/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[{'generated_text': "!\n\nTo open a current account in HDFC Bank, follow these steps:\n\n1. Visit the nearest HDFC Bank branch: You can find the nearest branch by visiting the HDFC Bank website and using the branch locator tool.\n2. Collect the account opening forms: You can either collect the account opening forms from the branch or download them from the HDFC Bank website.\n3. Fill the forms: Fill the account opening forms with accurate and complete information. You will need to provide personal and financial information, such as your name, address, date of birth, income, and employment details.\n4. Attach documents: Attach the required documents, such as your identity proof, address proof, and proof of income, to the account opening forms.\n5. Submit the forms: Submit the filled and signed account opening forms and documents to the branch.\n6. Verification: The bank will verify your information and documents.\n7. Account opening: Once your account is opened, you will receive a welco

In [0]:
def result_llm(string):
    results=gen_text(string)
    prompt="""Translate the given text to Hindi langauge
                 '''{}'''
      """. format(results)

    results_1= gen_text(prompt)
    return(results_1)

In [0]:
prompt="""Translate the given text to Hindi langauge
                 '''{}'''
      """. format(results)

In [0]:
results = gen_text([val])
print(results)

['What is the interest rate of hdfc bank?']
["\nHDFC Bank offers a wide range of interest rates on its deposit products, including savings accounts, fixed deposits, and recurring deposits. The interest rates offered by HDFC Bank are subject to change from time to time, and they may vary based on the deposit amount, tenure, and other factors.\n\nAs of March 2023, the interest rates offered by HDFC Bank on its savings accounts are as follows:\n\n* Balance up to ₹50 lakh: 3.50% p.a.\n* Balance above ₹50 lakh: 4.00% p.a.\n\nThe interest rates offered by HDFC Bank on its fixed deposits are as follows:\n\n* Tenure: 7 days to 10 years\n* Deposit amount: ₹1 lakh to ₹1 crore\n* Interest rate: 3.50% p.a. to 6.50% p.a.\n\nThe interest rates offered by HDFC Bank on its recurring deposits are as follows:\n\n* Tenure: 12 months to 120 months\n* Deposit amount: ₹1 lakh to ₹1 crore\n* Interest rate: 3.50% p.a. to 6.50% p.a.\n\nIt's important to note that these interest rates are subject to change, and

In [0]:
results=['A large language model is a type of artificial intelligence (AI) model that is trained on a large dataset of text to generate language outputs that are coherent and natural-sounding']

In [0]:
'Translate the given text to Hindi langauge'+" "+results[0]

'Translate the given text to Hindi langauge A large language model is a type of artificial intelligence (AI) model that is trained on a large dataset of text to generate language outputs that are coherent and natural-sounding'

In [0]:
results_1 = gen_text(["""Translate the below text to Hindi langauge
                 '''The current interest rate offered by HDFC Bank on home loans varies based on several factors such as loan amount, loan tenure, and credit score. However, as of March 2023, the starting interest rate for HDFC Bank home loans is around 8.75% per annum for women borrowers and around 9.25% per annum for male borrowers. It is advisable to check the current interest rates and other terms and conditions on the HDFC Bank website or by visiting a nearby HDFC Bank branch'''
"""])

["Translate the below text to Hindi langauge\n                 '''The current interest rate offered by HDFC Bank on home loans varies based on several factors such as loan amount, loan tenure, and credit score. However, as of March 2023, the starting interest rate for HDFC Bank home loans is around 8.75% per annum for women borrowers and around 9.25% per annum for male borrowers. It is advisable to check the current interest rates and other terms and conditions on the HDFC Bank website or by visiting a nearby HDFC Bank branch'''\n"]


/databricks/python/lib/python3.10/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [0]:
print(results_1)

['\n                 के लिए हड़ताल होम लोन के लिए हड़ताल बैंक द्वारा सर्विस के लिए कई संभावनाएं हैं, जैसे लोन संख्या, लोन तिथि और क्रेडिट सकोर. परंतु, मार्च 2023 में, HDFC बैंक होम लोन के लिए सर्विस के लिए अपने स्तर के दाम होंगे, जहां महिलाओं के लिए उम्मीद दाम 8.75% प्रति वर्ष होगा और मężczyzn के लिए 9.25% प्रति वर्ष होगा. यह आवश्यक है कि हड़ताल बैंक के ऑफिसियल वेबसाइट पर या नजीब हड़ताल बैंक ब्रांच का दौरा करके ताजा दाम और अन्य संबधीय शर्तें की ताजा की जाएं.']


In [0]:
results_1 = gen_text(["एचडीएफसी बैंक की वर्तमान ब्याज दर क्या है?"])
print(results_1[0])

/databricks/python/lib/python3.10/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(




Answer: The current interest rate offered by HDFC Bank on home loans varies based on several factors such as loan amount, loan tenure, and credit score. However, as of March 2023, the starting interest rate for HDFC Bank home loans is around 8.75% per annum for women borrowers and around 9.25% per annum for male borrowers. It is advisable to check the current interest rates and other terms and conditions on the HDFC Bank website or by visiting a nearby HDFC Bank branch.


In [0]:
results_1 = gen_text(["एचडीएफसी बैंक में खाता खोलने की प्रक्रिया क्या है? हिंदी में उत्तर दीजिए"])
print(results_1[0])

['एचडीएफसी बैंक में खाता खोलने की प्रक्रिया क्या है? हिंदी में उत्तर दीजिए']
!

To open a current account in HDFC Bank, follow these steps:

1. Visit the nearest HDFC Bank branch: You can find the nearest branch by visiting the HDFC Bank website and using the branch locator tool.
2. Collect the account opening forms: You can either collect the account opening forms from the branch or download them from the HDFC Bank website.
3. Fill in the forms: Fill in the account opening forms with your personal and financial information. You will need to provide your name, address, date of birth, income details, and other relevant information.
4. Attach documents: Attach the required documents such as your identity proof, address proof, and proof of income. The bank may also require additional documents based on your occupation or business type.
5. Submit the forms and documents: Submit the filled-in forms and documents to the branch. The bank's representative will review your application and verif

In [0]:
results_2 = gen_text(["""Translate the given text to Hindi langauge
                 '''To open a current account in HDFC Bank, you can follow these steps:
1. Visit the nearest HDFC Bank branch: You can find the nearest branch by visiting the HDFC Bank website and using the branch locator tool.
2. Collect the account opening forms: You can collect the account opening forms and other required documents from the branch or download them from the HDFC Bank website.
'''
"""], max_new_tokens=1000)

print(results_2[0])


                 # Step 1: Extract the text to be translated
                 text = '''To open a current account in HDFC Bank, you can follow these steps:
1. Visit the nearest HDFC Bank branch: You can find the nearest branch by visiting the HDFC Bank website and using the branch locator tool.
2. Collect the account opening forms: You can collect the account opening forms and other required documents from the branch or download them from the HDFC Bank website.'''

                 # Step 2: Translate the text to Hindi
                 translation = translate(text, src='en', dest='hi')

                 # Step 3: Print the translated text
                 print(translation)

                 # Output
                 # हDFC बैंक में CURRENT खाता खोलने के लिए यह करें:
                 # 1. HDFC बैंक के निर्विहीन ब्रांच को विजित करें: हDFC बैंक के वेबसाइट पर विजित करें और ब्रांच लोकेटर टूल का उपयोग करें.
                 # 2. खाता खोलने के लिए फॉर्म संज्ञा करें: ब्रांच से या हDFC बैंक क

In [0]:
###Make end-to-end
##1. relevalant to hdfc bank (couple of queries) and banking
##2. general query, non banking (hypertension, commom people ask , not technical)
##Test out the pipeline